In [1]:
import requests
from dotenv import load_dotenv
import pymongo
import time
import uuid
import sys
import os
import importlib
import re
import json
import csv

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import const
#importlib.reload(const)    #if we update the file const.py

In [2]:
load_dotenv() 
GENERATIVE_URI = os.environ['GENERATIVE_URI']

In [3]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']
collection = db['tb_psm_1']

In [ ]:
#PSM I: 80 questions (70 multi choice, 5 multi select, 5 true false)
def generate_questions():
    context = 'PSM (Professional Scrum Master) I includes questions from the following Focus Areas as defined in the Professional Scrum Competencies: "Understanding and Applying the Scrum Framework: Empiricism, Scrum Values, Scrum Team, Events, Artifacts, Done. Developing People and Teams: Self-Managing Teams, Facilitation, Coaching. Managing Products with Agility: Forecasting & Release Planning, Product Value, Product Backlog Management, Stakeholders & Customers."'
    #multiple choice
    #text_prompt = 'Generate 10 high-quality multiple-choice questions with answers and explanations for the PSM I examination. Each question has more than 60 words in length and should focus more on complex real-world scenarios rather than definitions. Please provide a response in a structured JSON format with the key name "questions", including all explanations for each answer as a JSON object. Each explanation has more than 50 words. Sample response structure should like this: { "question" : "xxx", "options" : { "A" : "a text", "B" : "a text", "C" : "a text", "D" : "a text"},"answer" : "B","explanation" : { "A" : "a text", "B" : "a text", "C" : "a text", "D" : "a text"},"type" : "multiple-choice"}. Response should avoid error while parsing JSON format "Error decoding JSON: Expecting property name enclosed in double quotes".'
    #true/false
    #text_prompt = 'Generate 4 high-quality true-false questions (50% true, 50% false) with answers and explanations for the PSM I examination. Each question has more than 60 words in length and should focus more on complex real-world scenarios rather than definitions. Please provide a response in a structured JSON format with the key name "questions", including all explanations for each answer as a JSON object. Each explanation has more than 50 words. Sample response structure should like this: { "question" : "xxx", "options" : { "A" : "True", "B" : "False"},"answer" : "A","explanation" : { "A" : "a text", "B" : "a text"},"type" : "true-false"}. Response should avoid error while parsing JSON format "Error decoding JSON: Expecting property name enclosed in double quotes".'
    #multi selection
    text_prompt = 'Generate 6 high-quality multiple-selection questions with answers and explanations for the PSM I examination. Each question has more than 60 words in length and should focus more on complex real-world scenarios rather than definitions. Please provide a response in a structured JSON format with the key name "questions", including all explanations for each answer as a JSON object. Each explanation has more than 50 words. Sample response structure should like this: { "question" : "xxx", "options" : { "A" : "a text", "B" : "a text", "C" : "a text", "D" : "a text", "E": "a_text"},"answer" : [],"explanation" : { "A" : "a text", "B" : "a text", "C" : "a text", "D" : "a text", "E" : "a text"},"type" : "multiple-selection"}. "answer" should have 2 to 4 correct elements.Response should avoid error while parsing JSON format "Error decoding JSON: Expecting property name enclosed in double quotes".'
    #
    raw_generated_text = const.post_request_generative_ai(context + text_prompt)

    #print(raw_generated_text)
    questions = const.extract_questions_from_candidates(raw_generated_text)
    if questions:
        #parse questions and answers
        for q in questions:
            q['exported'] = 0
            q['uuid'] = const.generate_random_uuid()
            #print(q)
            const.insert_questions(collection, q)
    else:
        print(raw_generated_text)
        print("No questions found in the parsed content")
#test
for i in range(10):
    #generate_questions() #10 sentences 30 secs
    print('===== Finish loop: ' + str(i))

===== Finish loop: 0
===== Finish loop: 1
===== Finish loop: 2
===== Finish loop: 3
===== Finish loop: 4
===== Finish loop: 5
===== Finish loop: 6
===== Finish loop: 7
===== Finish loop: 8
===== Finish loop: 9


In [18]:
#unify fields in all documents
def unify_fields():
    #questionType, question_type -> type
    # correctAnswer, correctAnswers -> answer
    # answers -> options
    all_docs = collection.find({'exported':0, 'type':'multiple-choice', 'explanation':None})
    for doc in all_docs:
        hasUpdated = False
        update_doc = {}
        #print(doc['uuid'])
        # if 'type' not in doc:
        #     if 'questionType' in doc:
        #         hasUpdated = True
        #         update_doc['type'] = doc['questionType']
        #     elif 'question_type' in doc:
        #         hasUpdated = True
        #         update_doc['type'] = doc['question_type']
        #     else:
        #         hasUpdated = True
        #         update_doc['type'] = 'multiple-choice'
        # if 'answer' not in doc:
        #     if 'correctAnswer' in doc:
        #         hasUpdated = True
        #         update_doc['answer'] = doc['correctAnswer']
        #     elif 'correctAnswers' in doc:
        #         hasUpdated = True
        #         update_doc['answer'] = doc['correctAnswers']
        # if 'options' not in doc:
        #     if 'answers' in doc:
        #         hasUpdated = True
        #         update_doc['options'] = doc['answers']

        #if hasUpdated:
        #print('a')
        #collection.update_one({'uuid': doc['uuid']}, {'$set': {'explanation': doc['explanations']}})
    #update 2
    # all_docs = collection.find({'filename': 'aws_pmp_test_6_20250506.csv'})
    # for doc in all_docs:
    #     collection.update_one({'uuid': doc['uuid']}, {'$set': {'exported': 0, 'filename': ''}})

#test
#unify_fields()

In [ ]:
#Question,Question Type,Answer Option 1,Explanation 1,Answer Option 2,Explanation 2,Answer Option 3,Explanation 3,Answer Option 4,Explanation 4,Answer Option 5,Explanation 5,Answer Option 6,Explanation 6,Correct Answers,Overall Explanation,Domain
def export_csv(path, filename):
    #get questions that not exported yet. Note that: each part must follow by domain percents
    file_data = []
    #append header line (both multi-choice and multi-selection)
    file_data.append(['Question','Question Type','Answer Option 1','Explanation 1','Answer Option 2','Explanation 2','Answer Option 3','Explanation 3','Answer Option 4','Explanation 4','Answer Option 5','Explanation 5','Answer Option 6','Explanation 6','Correct Answers','Overall Explanation','Domain'])
    exported_uuid = []
    manual_uuid = []
    #1. export multiple-choice first
    pipeline = [
                {"$match": {'exported': 0, 'type': 'multiple-choice'}},
                {"$sample": {"size": 72}}
            ]
    random_documents = list(collection.aggregate(pipeline))
    for doc in random_documents:
        file_data.append([doc['question'].replace('  ', ' ').replace('\n', ''), 'multiple-choice', 
                                  doc['options']['A'], doc['explanation']['A'].replace('  ', ' ').replace('\n', ''),     #A
                                  doc['options']['B'], doc['explanation']['B'].replace('  ', ' ').replace('\n', ''),     #B
                                  doc['options']['C'], doc['explanation']['C'].replace('  ', ' ').replace('\n', ''),     #C
                                  doc['options']['D'], doc['explanation']['D'].replace('  ', ' ').replace('\n', ''),     #D
                                  '', '',   #E
                                  '', '',   #6
                                  const.map_index(doc['answer']), #correct answer
                                  '', #overall
                                  '' #domain
                                  ])
        exported_uuid.append(doc['uuid'])
    #2. true false questions
    pipeline = [
                {"$match": {'exported': 0, 'type': 'true-false'}},
                {"$sample": {"size": 4}}
            ]
    random_documents = list(collection.aggregate(pipeline))
    for doc in random_documents:
        file_data.append([doc['question'].replace('  ', ' ').replace('\n', ''), 'multiple-choice', 
                                  doc['options']['A'], doc['explanation']['A'].replace('  ', ' ').replace('\n', ''),     #A
                                  doc['options']['B'], doc['explanation']['B'].replace('  ', ' ').replace('\n', ''),     #B
                                  '', '',     #C
                                  '', '',     #D
                                  '', '',   #E
                                  '', '',   #6
                                  const.map_index(doc['answer']), #correct answer
                                  '', #overall
                                  '' #domain
                                  ])
        exported_uuid.append(doc['uuid'])
    #3. multi selection
    pipeline = [
                {"$match": {'exported': 0, 'type': 'multiple-selection'}},
                {"$sample": {"size": 4}}
            ]
    random_documents = list(collection.aggregate(pipeline))
    for doc in random_documents:
        exported_uuid.append(doc['uuid'])
        manual_uuid.append(doc['uuid']) #they do not suppor bulk upload this type of question, we need to manually add them
    #
    print('","'.join(manual_uuid))
    #save all questions to csv
    try:
        with open(path + filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(file_data)
            print(f"Data successfully saved to '{filename}'")
            for _id in exported_uuid:
                collection.update_one({'uuid': _id}, {'$set': {'exported': 1, 'filename': filename}})
    except Exception as e:
        print(f"An error occurred while saving the array: {e}")
    
#test|
export_csv('./psm_1_data/', 'aws_psm_I_test_6_20250506.csv')

40b715ad-4ec8-47d9-87c8-430bcbbc1e65","cdfc62cc-c8fc-4ca4-8a1f-c24f89f6246f","d09fdd5e-02d7-46ae-9d8f-c3d40439094f","dc42598c-059f-43ce-b970-d1c03ada98f0
Data successfully saved to 'aws_psm_I_test_6_20250506.csv'
